In [2]:
import pandas as pd
import numpy as np
import pickle
import os
from sklearn.preprocessing import LabelEncoder # <--- Added Import

# --- 1. Load Text Embeddings ---
print("Step 1: Loading Text Embeddings...")
# Check local path
pkl_path = 'MELD.Raw/text_emotion.pkl'

if not os.path.exists(pkl_path):
    print("❌ ERROR: 'text_emotion.pkl' not found.")
else:
    with open(pkl_path, 'rb') as f:
        text_data = pickle.load(f)

    # --- 2. Load Labels from CSV ---
    print("Step 2: Loading CSV Labels...")
    base_path = 'MELD.Raw'

    train_labels_df = pd.read_csv(os.path.join(base_path, 'train_sent_emo.csv'))
    dev_labels_df = pd.read_csv(os.path.join(base_path, 'dev_sent_emo.csv'))
    test_labels_df = pd.read_csv(os.path.join(base_path, 'test_sent_emo.csv'))

    # Helper to map IDs to Emotions
    def create_label_dict(df):
        label_dict = {}
        for _, row in df.iterrows():
            label_dict[(row['Dialogue_ID'], row['Utterance_ID'])] = row['Emotion']
        return label_dict

    train_label_dict = create_label_dict(train_labels_df)
    dev_label_dict = create_label_dict(dev_labels_df)
    test_label_dict = create_label_dict(test_labels_df)

    # --- 3. Extract IDs and Sync ---
    def extract_ids_and_labels(text_dict, label_dict):
        dia_ids = []
        utt_ids = []
        emotions = []

        for dia_id in text_dict.keys():
            num_utts = text_dict[dia_id].shape[0]
            for utt_idx in range(num_utts):
                key = (int(dia_id), utt_idx)
                if key in label_dict:
                    dia_ids.append(dia_id)
                    utt_ids.append(utt_idx)
                    emotions.append(label_dict[key])

        return dia_ids, utt_ids, emotions

    print("Step 3: Generating ID lists...")
    train_text_dict, val_text_dict, test_text_dict = text_data

    # Create the variables for matching with spectrograms
    train_dia_ids, train_utt_ids, train_emotions = extract_ids_and_labels(train_text_dict, train_label_dict)
    val_dia_ids, val_utt_ids, val_emotions = extract_ids_and_labels(val_text_dict, dev_label_dict)
    test_dia_ids, test_utt_ids, test_emotions = extract_ids_and_labels(test_text_dict, test_label_dict)

    # --- 4. ENCODE LABELS (CRITICAL ADDITION) ---
    print("Step 4: Encoding Labels...")
    le = LabelEncoder()
    # Fit on all emotions to ensure consistent mapping
    le.fit(train_emotions + val_emotions + test_emotions)

    # Create the 'y' variables required by the spectrogram script
    y_train = le.transform(train_emotions)
    y_val = le.transform(val_emotions)
    y_test = le.transform(test_emotions)

    print(f"✅ Success! Variables restored and encoded.")
    print(f"Train samples: {len(y_train)}")
    print(f"Classes: {le.classes_}")

C:\Users\shawn\AppData\Local\Temp\ipykernel_13708\2954950361.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Step 1: Loading Text Embeddings...
Step 2: Loading CSV Labels...
Step 3: Generating ID lists...
Step 4: Encoding Labels...
✅ Success! Variables restored and encoded.
Train samples: 9989
Classes: ['anger' 'disgust' 'fear' 'joy' 'neutral' 'sadness' 'surprise']
Step 3: Generating ID lists...
Step 4: Encoding Labels...
✅ Success! Variables restored and encoded.
Train samples: 9989
Classes: ['anger' 'disgust' 'fear' 'joy' 'neutral' 'sadness' 'surprise']


# Extract mp4 files from zips

# Convert audio to spectrogram

In [ ]:
import librosa
import numpy as np
import os
from tqdm import tqdm
from sklearn.preprocessing import minmax_scale

# --- 1. SET UP PATHS ---

# Define the paths where the MP4 files are located
ACTUAL_MP4_PATHS = {
    'Train': 'MELD.Raw/train_splits',
    'Validation': 'MELD.Raw/dev_splits_complete',
    'Test': 'MELD.Raw/output_repeated_splits_test'
}

print("Audio Source Paths:")
print(f"  Train:      {ACTUAL_MP4_PATHS['Train']}")
print(f"  Validation: {ACTUAL_MP4_PATHS['Validation']}")
print(f"  Test:       {ACTUAL_MP4_PATHS['Test']}")
print("-" * 70)

# --- 2. Spectrogram Conversion Function ---
def process_audio_to_spectrogram(file_path, max_time_steps=128, n_mels=128):
    """
    Converts raw audio (from .wav or .mp4) to a 2D Mel-Spectrogram image.
    Uses a fixed duration and consistent padding/truncation for CNN input (128x128x1).
    """
    try:
        # Load audio (limited to 3 seconds at 16kHz for consistent size)
        y, sr = librosa.load(file_path, sr=16000, duration=3.0)

        # Generate Mel Spectrogram
        mels = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)

        # Convert to Log-Scale (dB)
        mels_db = librosa.power_to_db(mels, ref=np.max)

        # Normalization using sklearn's minmax_scale (0 to 1)
        mels_db = minmax_scale(mels_db.flatten()).reshape(mels_db.shape)

        # Padding/Truncating to achieve max_time_steps (128 columns)
        current_width = mels_db.shape[1]
        if current_width < max_time_steps:
            padding = max_time_steps - current_width
            mels_db = np.pad(mels_db, ((0, 0), (0, padding)), mode='constant')
        else:
            mels_db = mels_db[:, :max_time_steps]

        # Add channel dimension (128, 128) -> (128, 128, 1)
        return mels_db[..., np.newaxis]

    except Exception as e:
        # Return zeros for files that can't be processed
        return np.zeros((n_mels, max_time_steps, 1))


# --- 3. ID-Based Processing (Ensures Label Alignment) ---
def find_and_process_optimized(dia_ids, utt_ids, split_name, base_path):
    """
    Generates spectrograms in the EXACT order specified by dia_ids and utt_ids.
    This ensures 1:1 correspondence with label arrays.
    
    - Missing audio files are replaced with zero spectrograms
    - AppleDouble files (._*) are automatically filtered out
    - Output array length exactly matches the label array length
    
    Parameters:
    -----------
    dia_ids : list
        List of dialogue IDs from the label processing step
    utt_ids : list
        List of utterance IDs from the label processing step
    split_name : str
        Name of the split (Train/Validation/Test) for logging
    base_path : str
        Base directory containing the audio files
    
    Returns:
    --------
    np.ndarray
        Array of spectrograms with shape (len(dia_ids), 128, 128, 1)
    """
    print(f"\nIndexing audio files for {split_name} set from: {base_path}")

    # Index all audio files
    audio_file_index = {}

    if not os.path.exists(base_path):
        print(f"Error: Base path {base_path} does not exist!")
        return np.array([])

    # Walk through directory and index all .mp4 and .wav files
    # Filter out AppleDouble files (._*) which are Mac filesystem artifacts
    file_count = 0
    for root, _, files in os.walk(base_path):
        for file in files:
            # Skip AppleDouble files
            if file.startswith('._'):
                continue
            if file.endswith(('.mp4', '.wav')):
                audio_file_index[file] = os.path.join(root, file)
                file_count += 1

    print(f"Indexed {file_count} audio files")

    spectrograms = []
    missing_count = 0

    print(f"Generating spectrograms for {len(dia_ids)} utterances...")

    for d_id, u_id in tqdm(zip(dia_ids, utt_ids), total=len(dia_ids), desc=f"Processing {split_name}"):
        # MELD filenames: diaX_uttY.mp4
        filename_mp4 = f"dia{d_id}_utt{u_id}.mp4"
        filename_wav = f"dia{d_id}_utt{u_id}.wav"

        path = None
        if filename_mp4 in audio_file_index:
            path = audio_file_index[filename_mp4]
        elif filename_wav in audio_file_index:
            path = audio_file_index[filename_wav]

        if path:
            spec = process_audio_to_spectrogram(path)
            spectrograms.append(spec)
        else:
            missing_count += 1
            # Append zero spectrogram to maintain index alignment with labels
            spectrograms.append(np.zeros((128, 128, 1)))

    if missing_count > 0:
        print(f"Warning: {missing_count} audio files not found - zero spectrograms inserted")

    return np.array(spectrograms)


# --- 4. Generate Spectrograms for All Splits ---
print("\n" + "=" * 70)
print("STARTING SPECTROGRAM GENERATION")
print("=" * 70)
print("\nNote: This cell requires label variables from the previous cell:")
print("  train_dia_ids, train_utt_ids, val_dia_ids, val_utt_ids,")
print("  test_dia_ids, test_utt_ids\n")

# Process Train Set
X_audio_train = find_and_process_optimized(
    train_dia_ids, train_utt_ids, "Train", ACTUAL_MP4_PATHS['Train']
)

# Process Validation Set
X_audio_val = find_and_process_optimized(
    val_dia_ids, val_utt_ids, "Validation", ACTUAL_MP4_PATHS['Validation']
)

# Process Test Set
X_audio_test = find_and_process_optimized(
    test_dia_ids, test_utt_ids, "Test", ACTUAL_MP4_PATHS['Test']
)

print(f"\n{'=' * 70}")
print(f"SPECTROGRAM GENERATION COMPLETE")
print(f"{'=' * 70}")
print(f"\nFinal Shapes:")
print(f"  Train: {X_audio_train.shape}")
print(f"  Val:   {X_audio_val.shape}")
print(f"  Test:  {X_audio_test.shape}")

# --- 5. Save Arrays ---
SAVE_DIR = 'MELD_spectrograms/'
os.makedirs(SAVE_DIR, exist_ok=True)

print(f"\nSaving to: {SAVE_DIR}")

np.save(f"{SAVE_DIR}X_audio_train.npy", X_audio_train)
np.save(f"{SAVE_DIR}X_audio_val.npy", X_audio_val)
np.save(f"{SAVE_DIR}X_audio_test.npy", X_audio_test)

print("✓ All arrays saved successfully!")


Verified MP4 Source Path for Train: /content/meld_audio_splits/train
--------------------------------------------------
Step 2: Checking for and extracting audio archives...
Extracting train.tar.gz to /content/meld_audio_splits/train...


/tmp/ipython-input-878588619.py:57: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path=dest_path)


✅ Successfully extracted train.tar.gz.
Extracting dev.tar.gz to /content/meld_audio_splits/dev/dev_splits_complete...
✅ Successfully extracted dev.tar.gz.
Extracting test.tar.gz to /content/meld_audio_splits/test...
✅ Successfully extracted test.tar.gz.
Extraction check complete.
--------------------------------------------------

🎯 STARTING SYNCHRONIZED SPECTROGRAM GENERATION

⚠️  CRITICAL: This cell MUST be run AFTER the label processing cell!
   The spectrogram generation uses the filtered ID lists as input.
   This ensures perfect correspondence between audio and labels.

Indexing audio files for Train set from: /content/meld_audio_splits/train
Indexed 9989 audio/video files in the local Colab disk.
Generating spectrograms for 9989 utterances...
  → Processing ONLY the IDs from the filtered label set
  → This ensures 1:1 correspondence with y_train


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
Processing Train:  47%|████▋     | 4741/9989 [18:59<18:18,  4.78it/s]/tmp/ipython-input-878588619.py:75: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, sr=16000, duration=3.0)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
Processing Train:  47%|████▋     | 4742/9989 [18:59<16:35,  5.27it/s]/tmp/ipython-input-878588619.py:75: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(fi